# udn parser to mongodb

In [1]:
import re
import pymongo
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pymongo import MongoClient
%matplotlib inline

### Url set, then use bs4 to get page

In [2]:
url = "https://search.udn.com/search/searchNews4utf8.jsp?ch=udn.all&df=2&rc=1000&wc=80000&pw=220&mc=&q=%25E6%25AF%2594%25E7%2589%25B9%25E5%25B9%25A3"
request = requests.get(url)
content = request.content
soup = BeautifulSoup(content, "html.parser")

In [3]:
# text in html
text = soup.contents[4].split(';')
text = text[1:]
text

['\r\n\r\narr[0] = new Array(8)',
 "\r\narr[0][0] = 'https://fund.udn.com/fund/story/5854/2702518'",
 "\r\narr[0][1] = 'https://s.udn.com.tw/static/img/nophoto2.jpg'",
 "\r\narr[0][2] = '比特幣概念股 短線有壓'",
 "\r\narr[0][3] = '基金'",
 "\r\narr[0][4] = '2017/09/14'",
 "\r\narr[0][5] = '大陸監管出重手，比特幣交易平台將全線關閉，概念股後市恐受衝擊，包括主機板、繪圖卡及少數的IC設計股等，股價有回檔壓力。台新投顧協理黃文清認為，對台灣供應鏈而言不致於出現明顯影響，因各業者各自有營運主軸，比特幣占的比率不大，不致於影響相關個股的業績表現。'",
 "\r\narr[0][6] = ''",
 "\r\narr[0][7] = '記者張瑞益／台北報導'",
 '\r\n\r\narr[1] = new Array(8)',
 "\r\narr[1][0] = 'https://fund.udn.com/fund/story/5858/2682484'",
 "\r\narr[1][1] = 'https://uc.udn.com.tw/photo/2017/09/04/2/3959040.jpg'",
 "\r\narr[1][2] = '墨比爾斯：加密貨幣將遭政府打壓 投資人將擁抱黃金'",
 "\r\narr[1][3] = '基金'",
 "\r\narr[1][4] = '2017/09/04'",
 "\r\narr[1][5] = '有「新興市場教父」之稱的富蘭克林坦伯頓新興市場研究團隊總裁墨比爾斯（MARK MOBIUS），在加密貨幣的爆炸性成長中，嗅到風險。墨比爾斯4日表示，各國政府將開始取締虛擬貨幣，因為虛擬貨幣被用於非法融資，且恐怖組織與販毒集團助長了其漲勢。他說：「加密貨幣正在失控，也將引起各國政府的關注。投資人將回頭擁抱黃金，因為人們懷疑自己究竟能否信任這些虛擬貨幣。」取締虛擬貨幣的行動已在中國大陸開跑，當地恰好是比特幣礦工的主要聚集地。在美國，無論銀行與監管機構都正在研究加密貨幣的

### clean data with regulation expression 

In [4]:
title_list = []
link_list = []
date_list = []
content_list = []

In [5]:
for item in text:
    if re.search(r'arr\[\d+\]\[0\]', item):
        link = re.search(r'arr\[\d+\]\[0\]', item).string.split('=')[1][2:-1]  #[2:-1] for removing''
    if re.search(r'arr\[\d+\]\[2\]', item):
        title = re.search(r'arr\[\d+\]\[2\]', item).string.split('=')[1][2:-1]
    if re.search(r'arr\[\d+\]\[4\]', item):
        date = re.search(r'arr\[\d+\]\[4\]', item).string.split('=')[1][2:-1]
    if re.search(r'arr\[\d+\]\[5\]', item):
        content = re.search(r'arr\[\d+\]\[5\]', item).string.split('=')[1][2:-1]
        #print(title, link, date, content)
        title_list.append(title)
        link_list.append(link)
        date_list.append(date)
        content_list.append(content)

In [6]:
# check length the same
print(len(title_list), len(link_list), len(date_list), len(content_list))

565 565 565 565


### to pd.dataframe

In [7]:
# put data in dict and pd.df
d = {"Title": title_list, "Link": link_list, "Date": date_list, "Content": content_list, "Source": 'udn'}

In [8]:
df = pd.DataFrame(d)

### 整理時間格式

In [9]:
for index, row in df.iterrows():
    x = datetime.datetime.strptime(row['Date'], '%Y/%m/%d')
    df.set_value(index, 'Date', x)

In [10]:
# remove last ones without entire one-date data
start_date = '2017-01-01 00:00:00'
end_date  = '2017-09-17 00:00:00'

start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date  = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

df = df[(df.Date > start_date)]
df.tail()

,Content,Date,Link,Source,Title
468,虛擬貨幣比特幣（BITCOIN）5日盤中在短短四小時內暴跌20%，可能寫下兩年來最差單日表現...,2017-01-05 00:00:00,http://udn.com/news/story/5/2212646,udn,前市才創新高 比特幣5日崩跌20%
469,比特幣今天升值至逼近歷史高點，在全球致力對抗經濟不確定性升高時，這種飛速成長的數位貨幣，成為...,2017-01-05 00:00:00,http://udn.com/news/story/5/2212597,udn,比特幣持續攀升 突破1100美元大關
470,比特幣價格周三創下歷史新高，反映因中國等地外匯管制，漸漸更多人接納這個貨幣。這個問世八年的數...,2017-01-05 00:00:00,http://udn.com/news/story/6811/2210898,udn,比特幣再創歷史新高 反映外匯管制引發恐慌
471,"虛擬貨幣比特幣（BITCOIN）近期價格快速飆漲重回1,000美元，今日報價再度走高來到1,...",2017-01-04 00:00:00,http://udn.com/news/story/6/2208758,udn,中印最愛比特幣 身價飆破1千美元
472,比特幣今天升破1000美元，創至少3年來最佳水準，持續不停的快速漲勢讓比特幣成為2016年表...,2017-01-03 00:00:00,http://udn.com/news/story/5/2206478,udn,2016年表現最佳貨幣 比特幣奪冠


### to list，用於存入db

In [11]:
tt = []
for index, row in df.iterrows():
    temp = {
        "Date": row['Date'],
        "Link": row['Link'],
        "Source": row['Source'],
        "Title": row['Title'],
        "Content": row['Content']
    }
    tt.append(temp)

### to mongodb

In [12]:
client = MongoClient('mongodb://localhost:27017/')
db = client.bitcoinnews_db
for item in tt:
    db.newsCollection.insert_one(item)